In [1]:
# download the train and test data
! wget https://gitlab.com/brainekt_ai/kaggle-short-ans/-/raw/main/train.tsv
! wget https://gitlab.com/brainekt_ai/kaggle-short-ans/-/raw/main/test.csv

--2021-11-26 11:28:51--  https://gitlab.com/brainekt_ai/kaggle-short-ans/-/raw/main/train.tsv
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4291891 (4.1M) [text/plain]
Saving to: ‘train.tsv’

train.tsv           100%[===================>]   4.09M  19.3MB/s    in 0.2s    

2021-11-26 11:28:52 (19.3 MB/s) - ‘train.tsv’ saved [4291891/4291891]

--2021-11-26 11:28:52--  https://gitlab.com/brainekt_ai/kaggle-short-ans/-/raw/main/test.csv
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50272 (49K) [text/plain]
Saving to: ‘test.csv’

test.csv            100%[===================>]  49.09K  --.-KB/s    in 0.001s  

2021-11-26 11:28:53 (84.5 MB/s) - ‘test.csv’ saved [50

In [1]:
! nvidia-smi

Fri Nov 26 13:00:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:61:00.0 Off |                    0 |
| N/A   30C    P0    57W / 300W |   1150MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:62:00.0 Off |                    0 |
| N/A   30C    P0    43W / 300W |     11MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [2]:
! ls

bert_sa.pt  kaggle_short_answer_scoring.ipynb  test.csv  train.tsv


In [3]:
import pandas as pd
train = pd.read_csv('train.tsv', sep='\t')
test = pd.read_csv('test.csv')

In [4]:
train.head()

,Id,EssaySet,Score1,Score2,EssayText
0,1,1,1,1,Some additional information that we would need...
1,2,1,1,1,"After reading the expirement, I realized that ..."
2,3,1,1,1,"What you need is more trials, a control set up..."
3,4,1,0,0,The student should list what rock is better an...
4,5,1,2,2,For the students to be able to make a replicat...


In [5]:
test.head()

,id,essay_set,essay_weight
0,2230,1,1
1,2231,1,1
2,2232,1,1
3,2233,1,1
4,2234,1,1


In [6]:
import random
from sklearn.utils import shuffle
random.seed(19)

train_sa = []
train_scores = []
test_sa = []
test_scores = []

# shuffle
train = shuffle(train)
train.reset_index(inplace=True, drop=True)

for i in range(len(train)):
  if i <= len(train) * 0.8:
    train_sa.append(train["EssayText"][i])
    train_scores.append(train["Score1"][i])
  else:
    test_sa.append(train["EssayText"][i])
    test_scores.append(train["Score1"][i])   

In [7]:
print(len(train_sa))
print(len(train_scores))
print(len(test_sa))
print(len(test_scores))

13766
13766
3441
3441


In [8]:
print(set(train_scores))

{0, 1, 2, 3}


In [9]:
print(set(test_scores))

{0, 1, 2, 3}


In [10]:
max(len(a) for a in train_sa)

1819

In [11]:
import torch
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', max_length = 512, padding_side = 'right')

In [12]:
from transformers import BertForSequenceClassification

num_class = 4 # we have 4 score classes

bertclassifier = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels = 4)

In [13]:
from torch.utils.data import Dataset, DataLoader

# create our dataset

class ShortAnsDataset(Dataset):
    def __init__(self, texts, class_labels, tokenizer):
        self.texts = texts
        self.class_labels = class_labels
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.class_labels)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        encoded = tokenizer.encode_plus(self.texts[idx], add_special_tokens=True, max_length = 512, pad_to_max_length = True,
                                return_token_type_ids = False,
                                return_attention_mask = True)  # Batch size 1
            
        # text to tokens
        token = torch.tensor(encoded['input_ids']) #.unsqueeze(0)
        mask = torch.tensor(encoded['attention_mask'])
        # ohe label
        ohe = torch.tensor(self.class_labels[idx])

        return token, mask, ohe

In [14]:
dataset_train = ShortAnsDataset(train_sa, train_scores, tokenizer)
dataset_test = ShortAnsDataset(test_sa, test_scores, tokenizer)

# in colab, we can use high batch size

sa_dataloader_train = DataLoader(dataset_train, batch_size=32, shuffle=True) 
sa_dataloader_test = DataLoader(dataset_test, batch_size=4, shuffle=True)

In [15]:
import transformers
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # cuda for gpu acceleration

# optimizer

optimizer = transformers.AdamW(bertclassifier.parameters(), lr = 1e-5)


bertclassifier.to(device) # taking the model to gpu if possible

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [16]:
# training epochs

epochs = 5

# metrics

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score
import numpy as np

train_losses = []

train_metrics = {'acc': [], 'f1': [], 'kappa': []}
test_metrics = {'acc': [], 'f1': [], 'kappa': []}

# progress bar

from tqdm import tqdm_notebook

for e in tqdm_notebook(range(epochs)):
    train_loss = 0.0
    train_acc = 0.0
    train_f1 = 0.0
    train_kappa = 0.0
    batch_cnt = 0
    
    bertclassifier.train()
    
    print(f'epoch: {e+1}')
    
    for i_batch, (X, X_mask, y) in tqdm_notebook(enumerate(sa_dataloader_train)):
        X = X.to(device)
        X_mask = X_mask.to(device)
        y = y.to(device)
        
        
        optimizer.zero_grad()
        
        loss, y_pred = bertclassifier(X, attention_mask = X_mask, labels = y)

        train_loss += loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(bertclassifier.parameters(), 1.0)

        optimizer.step()
        
        y_pred = torch.argmax(y_pred, dim = -1)
        
        # update metrics
        train_acc += accuracy_score(y.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
        train_f1 += f1_score(y.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), average = 'micro')
        if np.array_equal(y.cpu().detach().numpy(), y_pred.cpu().detach().numpy()):
            train_kappa += 1.
        else:
            train_kappa += cohen_kappa_score(y.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), weights = 'quadratic')
        batch_cnt += 1
    
    print(f'train loss: {train_loss/batch_cnt}')
    print(f'acc: {train_acc/batch_cnt}')
    print(f'f1: {train_f1/batch_cnt}')
    train_losses.append(train_loss/batch_cnt)
    train_metrics['acc'].append(train_acc/batch_cnt)
    train_metrics['f1'].append(train_f1/batch_cnt)
    train_metrics['kappa'].append(train_kappa/batch_cnt)
        
        
    test_loss = 0.0
    test_acc = 0.0
    test_f1 = 0.0
    test_kappa = 0.0
    batch_cnt = 0
    
    bertclassifier.eval()
    with torch.no_grad():
        for i_batch, (X, X_mask, y) in enumerate(sa_dataloader_test):
            X = X.to(device)
            X_mask = X_mask.to(device)
            y = y.to(device)

            y_pred = bertclassifier(X, attention_mask = X_mask, labels = y)[1] # in eval model we get the softmax output so, don't need to index

            
            y_pred = torch.argmax(y_pred, dim = -1)

            # update metrics
            test_acc += accuracy_score(y.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
            test_f1 += f1_score(y.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), average = 'micro')
            if np.array_equal(y.cpu().detach().numpy(), y_pred.cpu().detach().numpy()):
                test_kappa += 1.
            else:
                test_kappa += cohen_kappa_score(y.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), weights = 'quadratic')
            batch_cnt += 1
            
    test_metrics['acc'].append(test_acc/batch_cnt)
    test_metrics['f1'].append(test_f1/batch_cnt)
    test_metrics['kappa'].append(test_kappa/batch_cnt)
    print(f'test acc: {test_acc/batch_cnt}')
    print(f'test f1: {test_f1/batch_cnt}')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


epoch: 1


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



train loss: 0.8935734247248577
acc: 0.6014597834493427
f1: 0.6014597834493427
test acc: 0.6608594657375145
test f1: 0.6608594657375145
epoch: 2


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...


train loss: 0.5577772878922885
acc: 0.7654920726991492
f1: 0.7654920726991492
test acc: 0.6977351916376306
test f1: 0.6977351916376306
epoch: 4


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



train loss: 0.46487432904973663
acc: 0.8120407965970611
f1: 0.8120407965970611
test acc: 0.7105110336817654
test f1: 0.7105110336817654
epoch: 5


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



train loss: 0.3633637853191513
acc: 0.8609097061098221
f1: 0.8609097061098221
test acc: 0.7157375145180023
test f1: 0.7157375145180023



In [17]:
torch.save(bertclassifier, "bert_sa.pt") 

In [19]:
y_trues = []
y_preds = []

bertclassifier.eval()
with torch.no_grad():
    for i_batch, (X, X_mask, y) in tqdm_notebook(enumerate(sa_dataloader_test)):
        X = X.to(device)
        X_mask = X_mask.to(device)
        y = y.to(device)

        y_pred = bertclassifier(X, attention_mask = X_mask, labels = y)[1] # in eval model we get the softmax output so, don't need to index

        y_pred = torch.argmax(y_pred, dim = -1)
        
        y_trues.extend(y.cpu().detach().numpy().tolist())
        y_preds.extend(y_pred.cpu().detach().numpy().tolist())

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [20]:
y_trues[:5]

[1, 3, 1, 0, 0]

In [21]:
y_preds[:5]

[1, 3, 1, 1, 0]

In [22]:
cohen_kappa_score(y_trues, y_preds, weights = 'quadratic')

0.7595809748343627